In [13]:
import pandas as pd
import numpy as np
import random 
import pickle
import pickle
from sklearn import preprocessing
from collections import deque

df = pd.read_csv("crypto_data/LTC-USD.csv", names=["time", "low", "high", "open", "close", "volume"])

df.head(10)

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978
5,1528968960,96.459999,96.519997,96.459999,96.519997,16.991997
6,1528969020,96.379997,96.529999,96.510002,96.440002,95.524078
7,1528969080,96.349998,96.529999,96.480003,96.470001,175.205307
8,1528969140,96.349998,96.550003,96.480003,96.400002,43.652802
9,1528969200,96.400002,96.430000,96.430000,96.400002,8.160000


In [14]:
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = "LTC-USD"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]

for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, names=["time", "low", "high", "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    df.set_index("time", inplace=True)
    
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    print(df.head())
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)

main_df.head(10)

            BTC-USD_close  BTC-USD_volume
time                                     
1528968660    6489.549805        0.587100
1528968720    6487.379883        7.706374
1528968780    6479.410156        3.088252
1528968840    6479.410156        1.404100
1528968900    6479.979980        0.753000
            LTC-USD_close  LTC-USD_volume
time                                     
1528968660      96.580002        9.647200
1528968720      96.660004      314.387024
1528968780      96.570000       77.129799
1528968840      96.500000        7.216067
1528968900      96.389999      524.539978
            ETH-USD_close  ETH-USD_volume
time                                     
1528968720      486.01001       26.019083
1528968780      486.00000        8.449400
1528968840      485.75000       26.994646
1528968900      486.00000       77.355759
1528968960      486.00000        7.503300
            BCH-USD_close  BCH-USD_volume
time                                     
1528968660     871.719971        5

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume
time,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500
1528968960,6480.000000,1.490900,96.519997,16.991997,486.00000,7.503300,869.989990,1.669014
1528969020,6477.220215,2.731950,96.440002,95.524078,485.98999,85.877251,869.450012,0.865200
1528969080,6480.000000,2.174240,96.470001,175.205307,485.98999,160.915192,869.989990,23.534929
1528969140,6479.990234,0.903100,96.400002,43.652802,485.98999,61.371887,870.000000,2.300000


In [15]:
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

main_df.head(10)

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,future
time,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,96.500000
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577,96.389999
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300,96.519997
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862,96.440002
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500,96.470001
1528968960,6480.000000,1.490900,96.519997,16.991997,486.00000,7.503300,869.989990,1.669014,96.400002
1528969020,6477.220215,2.731950,96.440002,95.524078,485.98999,85.877251,869.450012,0.865200,96.400002
1528969080,6480.000000,2.174240,96.470001,175.205307,485.98999,160.915192,869.989990,23.534929,96.400002
1528969140,6479.990234,0.903100,96.400002,43.652802,485.98999,61.371887,870.000000,2.300000,96.400002


In [6]:
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

main_df.head(20)

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,future,target
time,,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,96.500000,0
1528968720,6487.379883,7.706374,96.660004,314.387024,486.010010,26.019083,870.859985,26.856577,96.389999,0
1528968780,6479.410156,3.088252,96.570000,77.129799,486.000000,8.449400,870.099976,1.124300,96.519997,0
1528968840,6479.410156,1.404100,96.500000,7.216067,485.750000,26.994646,870.789978,1.749862,96.440002,0
1528968900,6479.979980,0.753000,96.389999,524.539978,486.000000,77.355759,870.000000,1.680500,96.470001,1
1528968960,6480.000000,1.490900,96.519997,16.991997,486.000000,7.503300,869.989990,1.669014,96.400002,0
1528969020,6477.220215,2.731950,96.440002,95.524078,485.989990,85.877251,869.450012,0.865200,96.400002,0
1528969080,6480.000000,2.174240,96.470001,175.205307,485.989990,160.915192,869.989990,23.534929,96.400002,0
1528969140,6479.990234,0.903100,96.400002,43.652802,485.989990,61.371887,870.000000,2.300000,96.400002,0


In [29]:
pickle_out = open("cypto_df.pickle", "wb")
pickle.dump(main_df, pickle_out)
pickle_out.close()

In [30]:
# Data Normalization

pickle_in = open("cypto_df.pickle", "rb")
main_df = pickle.load(pickle_in)

main_df.head(10)

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,future
time,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,96.500000
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577,96.389999
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300,96.519997
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862,96.440002
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500,96.470001
1528968960,6480.000000,1.490900,96.519997,16.991997,486.00000,7.503300,869.989990,1.669014,96.400002
1528969020,6477.220215,2.731950,96.440002,95.524078,485.98999,85.877251,869.450012,0.865200,96.400002
1528969080,6480.000000,2.174240,96.470001,175.205307,485.98999,160.915192,869.989990,23.534929,96.400002
1528969140,6479.990234,0.903100,96.400002,43.652802,485.98999,61.371887,870.000000,2.300000,96.400002


In [31]:
def preprocess_df(df):
    df = df.drop("future", 1)  

    for col in df.columns:  
        if col != "target":  
            df[col] = df[col].pct_change()  
            df.dropna(inplace=True)  
            df[col] = preprocessing.scale(df[col].values)  

    df.dropna(inplace=True)  

    sequential_data = []  
    prev_days = deque(maxlen=SEQ_LEN)  

    for i in df.values:  
        prev_days.append([n for n in i[:-1]])  
        if len(prev_days) == SEQ_LEN: 
            sequential_data.append([np.array(prev_days), i[-1]])  

    random.shuffle(sequential_data) 
    
    buys = []
    sells = []  

    for seq, target in sequential_data:  
        if target == 0:  
            sells.append([seq, target])  
        elif target == 1:  
            buys.append([seq, target])  

    random.shuffle(buys)  
    random.shuffle(sells)  

    lower = min(len(buys), len(sells))  

    buys = buys[:lower]  
    sells = sells[:lower]  

    sequential_data = buys+sells  
    random.shuffle(sequential_data)
    
    X = []
    y = []

    for seq, target in sequential_data:  
        X.append(seq)  
        y.append(target)  

    return np.array(X), y 

In [32]:
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]
print(last_5pct)

1534643640


In [33]:
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

In [34]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [35]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 0 validation: 0
Dont buys: 0, buys: 0
VALIDATION Dont buys: 0, buys: 0


In [37]:
import pandas as pd
from collections import deque
import random
import numpy as np
from sklearn import preprocessing

SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTC-USD"


def classify(current, future):
    if float(future) > float(current):  # if the future price is higher than the current, that's a buy, or a 1
        return 1
    else:  # otherwise... it's a 0!
        return 0


def preprocess_df(df):
    df = df.drop("future", 1)  # don't need this anymore.

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic. Those nasty NaNs love to creep in.

    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.


main_df = pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider
for ratio in ratios:  # begin iteration
    print(ratio)
    dataset = f"crypto_data/{ratio}.csv"  # get the full path to the file.
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
#print(main_df.head())  # how did we do??

main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

#print(main_df.head())

times = sorted(main_df.index.values)  # get the times
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]  # get the last 5% of the times

validation_main_df = main_df[(main_df.index >= last_5pct)]  # make the validation data where the index is in the last 5%
main_df = main_df[(main_df.index < last_5pct)]  # now the main_df is all the data up to the last 5%


def preprocess_df(df):
    df = df.drop("future", 1)  # don't need this anymore.

    for col in df.columns:  # go through all of the columns
        if col != "target":  # normalize all ... except for the target itself!
            df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
            df.dropna(inplace=True)  # remove the nas created by pct_change
            df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic.


    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.

    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!

    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

    lower = min(len(buys), len(sells))  # what's the shorter length?

    buys = buys[:lower]  # make sure both lists are only up to the shortest length.
    sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!


main_df = pd.DataFrame() # begin empty

ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # the 4 ratios we want to consider
for ratio in ratios:  # begin iteration

    ratio = ratio.split('.csv')[0]  # split away the ticker from the file-name
    dataset = f"crypto_data/{ratio}.csv"  # get the full path to the file.
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  # read in specific file

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # ignore the other columns besides price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
#print(main_df.head())  # how did we do??

main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

main_df.dropna(inplace=True)

## here, split away some slice of the future data from the main main_df.
times = sorted(main_df.index.values)
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

BTC-USD
LTC-USD
BCH-USD
ETH-USD


In [38]:
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [39]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 77922 validation: 3860
Dont buys: 38961, buys: 38961
VALIDATION Dont buys: 1930, buys: 1930


In [40]:
pickle_out = open("cypto_train_x.pickle", "wb")
pickle.dump(train_x, pickle_out)
pickle_out.close()

pickle_out = open("cypto_train_y.pickle", "wb")
pickle.dump(train_y, pickle_out)
pickle_out.close()

pickle_out = open("cypto_val_x.pickle", "wb")
pickle.dump(validation_x, pickle_out)
pickle_out.close()

pickle_out = open("cypto_val_y.pickle", "wb")
pickle.dump(validation_y, pickle_out)
pickle_out.close()